In [2]:
import numpy as np
import tensorflow as tf

sess= tf.Session()

In [3]:
mat= np.ones((6,4))*5
mat[0,0]=9
mat

array([[9., 5., 5., 5.],
       [5., 5., 5., 5.],
       [5., 5., 5., 5.],
       [5., 5., 5., 5.],
       [5., 5., 5., 5.],
       [5., 5., 5., 5.]])

### With numpy svd

In [4]:
u,s,v= np.linalg.svd(mat)

In [5]:
print('shapes of u, s & v:\n',u.shape, s.shape, v.shape)

shapes of u, s & v:
 (6, 6) (4,) (4, 4)


### With sklearn_Truncated SVD

In [6]:
from sklearn.decomposition import TruncatedSVD

mod= TruncatedSVD(n_components=3)
mod.fit(mat)

TruncatedSVD(algorithm='randomized', n_components=3, n_iter=5,
             random_state=None, tol=0.0)

In [7]:
x_trans = mod.fit_transform(mat)
x_trans

array([[ 1.22007331e+01,  2.67247281e+00, -3.89754130e-49],
       [ 9.97862276e+00, -6.53519595e-01,  8.48776309e-33],
       [ 9.97862276e+00, -6.53519595e-01, -6.86674458e-33],
       [ 9.97862276e+00, -6.53519595e-01, -6.86674458e-33],
       [ 9.97862276e+00, -6.53519595e-01,  2.62286304e-33],
       [ 9.97862276e+00, -6.53519595e-01,  2.62286304e-33]])

In [8]:
x_trans.shape

(6, 3)

In [9]:
trans= mod.transform(mat)# np.dot(mat, mod.components_.T)
trans

array([[ 1.22007331e+01,  2.67247281e+00,  3.44169138e-15],
       [ 9.97862276e+00, -6.53519595e-01,  3.44169138e-15],
       [ 9.97862276e+00, -6.53519595e-01,  3.44169138e-15],
       [ 9.97862276e+00, -6.53519595e-01,  3.44169138e-15],
       [ 9.97862276e+00, -6.53519595e-01,  3.44169138e-15],
       [ 9.97862276e+00, -6.53519595e-01,  3.44169138e-15]])

* **Fit & Fit_transform yields are equivalent in sklearn svd**

In [10]:
np.allclose(x_trans, trans)

True

### With `tf.linalg.SVD`

In [11]:
st,ut,vt= sess.run(tf.linalg.svd(mat, full_matrices=True))

In [12]:
print('shapes of ut, st & vt:\n',ut.shape, st.shape, vt.shape)

shapes of ut, st & vt:
 (6, 6) (4,) (4, 4)


**Comparing values**
* 1. sigma val from tf vs. sigma val from skl

In [13]:
st[:3]#sigma val from tf

array([2.54307383e+01, 3.04590712e+00, 3.53270803e-16])

In [14]:
mod.singular_values_#sigma val from skl

array([2.54307383e+01, 3.04590712e+00, 1.34203318e-32])

**Both yield Similar Sigma values**

In [15]:
np.allclose(st[:3], mod.singular_values_)

True

* 2. right singular vec from tf vs skl 

In [16]:
vt[:3,:].T#right singular vec from tf

array([[ 5.55527595e-01,  4.80065652e-01,  4.80065652e-01],
       [-8.31498101e-01,  3.20734007e-01,  3.20734007e-01],
       [ 1.66533454e-16, -7.07106781e-01,  7.07106781e-01],
       [ 5.55111512e-17, -4.08248290e-01, -4.08248290e-01]])

In [17]:
mod.components_#right singular vec from sklearn

array([[ 0.5555276 ,  0.48006565,  0.48006565,  0.48006565],
       [ 0.8314981 , -0.32073401, -0.32073401, -0.32073401],
       [ 0.        ,  0.81649658, -0.40824829, -0.40824829]])

In [18]:
np.linalg.svd(mat, full_matrices=False)[2]#right singular values from numpy

array([[-0.5555276 , -0.48006565, -0.48006565, -0.48006565],
       [ 0.8314981 , -0.32073401, -0.32073401, -0.32073401],
       [ 0.        , -0.57735027, -0.21132487,  0.78867513],
       [ 0.        , -0.57735027,  0.78867513, -0.21132487]])

In [19]:
np.allclose(vt.T[0,:], mod.components_[0])
#similar Sigma values for 1st col from tf & sklearn

True

In [20]:
np.allclose(vt.T[:3,:], mod.components_)
#dissimilar Sigma values othewise from both

False

* 3. left singular vec from tf vs. skl

In [21]:
ut[:,:3]#left singular vec from tf

array([[ 4.79763230e-01, -8.77397996e-01, -5.78784447e-18],
       [ 3.92384312e-01,  2.14556639e-01,  8.94427191e-01],
       [ 3.92384312e-01,  2.14556639e-01, -2.23606798e-01],
       [ 3.92384312e-01,  2.14556639e-01, -2.23606798e-01],
       [ 3.92384312e-01,  2.14556639e-01, -2.23606798e-01],
       [ 3.92384312e-01,  2.14556639e-01, -2.23606798e-01]])

In [22]:
from sklearn.utils.extmath import randomized_svd
usk,ssk,vsk= randomized_svd(mat, n_components=3, n_iter=5, random_state=None)

usk#left singular vec from tf

array([[ 4.79763230e-01,  8.77397996e-01,  1.42543000e-16],
       [ 3.92384312e-01, -2.14556639e-01,  8.93306378e-01],
       [ 3.92384312e-01, -2.14556639e-01, -2.48349806e-01],
       [ 3.92384312e-01, -2.14556639e-01, -1.81621243e-01],
       [ 3.92384312e-01, -2.14556639e-01, -2.31667665e-01],
       [ 3.92384312e-01, -2.14556639e-01, -2.31667665e-01]])

In [23]:
np.allclose(ut[:,:3], usk)#dissimilar left singular values from both

False

* 4. fit_transfrom from tf svd vs. skl svd

In [24]:
trans= mod.fit_transform(mat)
trans #skearn's fit_transform

array([[ 1.22007331e+01,  2.67247281e+00, -1.59130736e-47],
       [ 9.97862276e+00, -6.53519595e-01,  9.08954444e-32],
       [ 9.97862276e+00, -6.53519595e-01, -7.35359592e-32],
       [ 9.97862276e+00, -6.53519595e-01, -7.35359592e-32],
       [ 9.97862276e+00, -6.53519595e-01,  2.80882370e-32],
       [ 9.97862276e+00, -6.53519595e-01,  2.80882370e-32]])

In [25]:
ut[:,:3]*st[:3] #tf's fit_transform

array([[ 1.22007331e+01, -2.67247281e+00, -2.04467647e-33],
       [ 9.97862276e+00,  6.53519595e-01,  3.15975012e-16],
       [ 9.97862276e+00,  6.53519595e-01, -7.89937530e-17],
       [ 9.97862276e+00,  6.53519595e-01, -7.89937530e-17],
       [ 9.97862276e+00,  6.53519595e-01, -7.89937530e-17],
       [ 9.97862276e+00,  6.53519595e-01, -7.89937530e-17]])

In [27]:
np.allclose(trans, ut[:,:3]*st[:3])

False

In [29]:
st,ut,vt= tf.linalg.svd(mat, full_matrices=False)
sess.run(tf.matmul(ut[:,:3], tf.linalg.diag(st[:3])))#X_transform similar to mod.fit_trasform above

array([[ 1.22007331e+01, -2.67247281e+00, -2.04467647e-33],
       [ 9.97862276e+00,  6.53519595e-01,  3.15975012e-16],
       [ 9.97862276e+00,  6.53519595e-01, -7.89937530e-17],
       [ 9.97862276e+00,  6.53519595e-01, -7.89937530e-17],
       [ 9.97862276e+00,  6.53519595e-01, -7.89937530e-17],
       [ 9.97862276e+00,  6.53519595e-01, -7.89937530e-17]])